In [0]:
from IPython.display import clear_output

In [0]:
!pip install pycodestyle flake8 pycodestyle_magic
!pip install PyDrive
clear_output()

In [0]:
%load_ext pycodestyle_magic

In [0]:
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# kaggle API сегодня не работало, решение ошибки не гуглилось :(

downloaded = drive.CreateFile({'id': '1Nh6yZij2vSlZtkAgv_K5l0paFMjB7xLC'})
downloaded.GetContentFile('imdb_master.csv')

# Assignment 5

Build CNN model for sentiment analysis (binary classification) of IMDB Reviews (https://www.kaggle.com/utathya/imdb-review-dataset).
You can use data with label="unsup" for pretraining of embeddings. Here you are forbidden to use test dataset for pretraining of embeddings.  
Your quality metric is accuracy score on test dataset. Look at "type" column for  train/test split.  
You can use pretrained embeddings from external sources.  
You have to provide data for trials with different hyperparameter values.  

You have to beat following baselines:  
[3 points] acc = 0.75  
[5 points] acc = 0.8  
[8 points] acc = 0.9  

[2 points] for using unsupervised data  

In [0]:
%tensorflow_version 1.x

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [0]:
import nltk
import pickle
import logging
import warnings
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from string import punctuation
from collections import Counter
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.casual import TweetTokenizer
from keras import layers as L
from keras import backend as K
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.sequence import pad_sequences

In [0]:
warnings.filterwarnings('ignore')

In [0]:
tf.get_logger().setLevel(logging.ERROR)
tf.autograph.set_verbosity(3)

In [0]:
def reset_tf_session():
    curr_session = tf.get_default_session()
    if curr_session:
        curr_session.close()
    K.clear_session()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    K.set_session(s)

In [0]:
np.random.seed(42)

In [0]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
stopwords = set(stopwords.words('english')) | set(punctuation)

# Preprocessing

In [0]:
df = pd.read_csv('imdb_master.csv', encoding='iso-8859-1')

In [21]:
df.head()

,Unnamed: 0,type,review,label,file
0,0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


In [0]:
df_train = df.loc[(df['type'] == 'train') & (df['label'] != 'unsup')]
df_test = df.loc[(df['type'] == 'test') & (df['label'] != 'unsup')]
df_unsup = df.loc[df['label'] == 'unsup']

In [0]:
X_train = df_train['review']
X_test = df_test['review']
y_train = np.array([1 if label == 'pos' else 0 for label in df_train['label']])
y_test = np.array([1 if label == 'pos' else 0 for label in df_test['label']])
sentences = df_unsup['review']

In [0]:
del df
del df_train
del df_test
del df_unsup

In [0]:
!rm imdb_master.csv

Я использовала TweetTokenizer, так как он выделяет смайлы и прочие специфичные для интернет-речи токены, которые могут быть маркером сентимента.

In [0]:
tokenizer = TweetTokenizer()

In [0]:
X_train = [tokenizer.tokenize(text) for text in X_train]
X_test = [tokenizer.tokenize(text) for text in X_test]
sentences = [tokenizer.tokenize(text) for text in sentences]

In [0]:
lemmatizer = WordNetLemmatizer()

In [0]:
def pos_tagger(word):
    tag = nltk.pos_tag([word])[0][1]
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('V'):
        return wordnet.VERB
    return wordnet.NOUN

In [0]:
X_train = [[lemmatizer.lemmatize(word.lower(), pos=pos_tagger(word.lower()))
           for word in text if word not in stopwords] for text in X_train]

X_test = [[lemmatizer.lemmatize(word.lower(), pos=pos_tagger(word.lower()))
          for word in text if word not in stopwords] for text in X_test]

sentences = [[lemmatizer.lemmatize(word.lower(), pos=pos_tagger(word.lower()))
             for word in text if word not in stopwords] for text in sentences]

Сохраним лемматизованные тексты.

In [0]:
with open('X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)

with open('X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)

with open('sentences.pkl', 'wb') as f:
    pickle.dump(sentences, f)

In [0]:
uploaded = drive.CreateFile({'title': 'X_train.pkl'})
uploaded.SetContentFile('X_train.pkl')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'X_test.pkl'})
uploaded.SetContentFile('X_test.pkl')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'sentences.pkl'})
uploaded.SetContentFile('sentences.pkl')
uploaded.Upload()

In [0]:
# код, скачивающий предобработанные тексты

# downloaded = drive.CreateFile({'id': '1m19e115PCIn5inLicGAZwldd_keokzKo'})
# downloaded.GetContentFile('X_train.pkl')

# downloaded = drive.CreateFile({'id': '1TtZIulmPGnyd7NQ_Ps6CBgl2NimfxbgA'})
# downloaded.GetContentFile('X_test.pkl')

# downloaded = drive.CreateFile({'id': '1H2MdO9_uQM1vDSMle50NCDQtAoa8evH_'})
# downloaded.GetContentFile('sentences.pkl')

# with open('X_train.pkl', 'rb') as f:
#    X_train = pickle.load(f)

# with open('X_test.pkl', 'rb') as f:
#    X_test = pickle.load(f)

# with open('sentences.pkl', 'rb') as f:
#    sentences = pickle.load(f)

Посчитаем количество уникальных слов в текстах, на которых будем обучать Word2Vec. Создадим словарь (и соответствующий обратный словарь), где ключи – слова, а значения – их индексы (чем частотнее слово, тем меньше значение индекса, максимальный индекс – у токена для незнакомых слов). Индексация с единицы (0 – для padding token).

In [27]:
word_counts = Counter(chain(*sentences)).most_common()
vocab = {word_counts[i][0]: i + 1 for i in range(len(word_counts))}
vocab_inv = {index: word for word, index in vocab.items()}
vocab_size = len(vocab) + 1
vocab_size

137234

Посчитаем максимальную и среднюю длину текста.

In [28]:
max(list(map(len, X_train)) +
    list(map(len, X_train)) +
    list(map(len, X_train)))

1514

In [29]:
np.mean(list(map(len, X_train)) +
        list(map(len, X_train)) +
        list(map(len, X_train)))

136.47684

Преобразуем тексты в обучающем и тестовом датасете в последовательности чисел.

In [0]:
X_train = [[vocab[word] if word in vocab else vocab_size for word in text]
           for text in X_train]

X_test = [[vocab[word] if word in vocab else vocab_size for word in text]
          for text in X_test]

Делать padding до максимальной длины неоптимально, так как она в несколько раз выше средней длины. Будем делать padding до 150 слов – это чуть-чуть больше средней длины текста.

In [0]:
X_train = pad_sequences(X_train, maxlen=150, padding='post')
X_test = pad_sequences(X_test, maxlen=150, padding='post')

# Word2Vec training

In [0]:
w2v = Word2Vec(sentences, negative=5, size=100, iter=100, sg=1)

In [0]:
w2v.wv.most_similar('film')

[('movie', 0.841327428817749),
 ('certainly', 0.6642634868621826),
 ('probably', 0.662882387638092),
 ('picture', 0.6517543196678162),
 ('cinema', 0.6354538202285767),
 ('one', 0.6313589811325073),
 ('flick', 0.6308745741844177),
 ('masterpiece', 0.6152960062026978),
 ('however', 0.6113929748535156),
 ('low-budget', 0.6112046241760254)]

Так как модель больше не будет обучаться, есть смысл вызвать метод init_sims(посчитать L2-нормализованные векторы модели заранее), чтобы снизить объём памяти, занимаемый моделью.

In [0]:
w2v.wv.init_sims(replace=True)

In [0]:
w2v.save('w2v.model')

In [0]:
uploaded = drive.CreateFile({'title': 'w2v.model'})
uploaded.SetContentFile('w2v.model')
uploaded.Upload()

In [0]:
# код, скачивающий готовую модель

# downloaded = drive.CreateFile({'id': '1zDhxIA2BaVV7bUy7Wyrn9PXwE7g0RnSK'})
# downloaded.GetContentFile('w2v.model')

# w2v = Word2Vec.load('w2v.model')

Создадим матрицу эмбеддингов. Эмбеддинг для padding token – вектор длины 100, состоящий из нулей. Эмбеддинги для незнакомых слов состоят из случайных чисел в диапазоне от -0.25 до 0.25.

In [0]:
embedding_matrix = np.array([np.zeros(100)] +
                            [w2v[vocab_inv[i]] if vocab_inv[i] in w2v
                             else np.random.uniform(-0.25, 0.25, 100)
                             for i in range(1, vocab_size)])

# CNN training
### Hyperparameter tuning
Обучались шесть конволюционных нейросетей, отличающихся методом регуляризации – CNN без регуляризации, CNN с нормализацией батчей, CNN с дропаутом с вероятностью 0.1, CNN с дропаутом с вероятностью 0.5, CNN с дропаутом с вероятностью 0.8 и CNN с нормализацией батчей и дропаутом с той вероятностью, которая давала лучшие результаты.
### Basic CNN model

In [0]:
reset_tf_session()

In [35]:
model = Sequential()
model.add(L.Embedding(vocab_size, 100, weights=[embedding_matrix],
                      input_length=150, trainable=False))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))

model.add(L.Flatten())
model.add(L.Dense(128, activation='relu'))
model.add(L.Dense(64, activation='relu'))
model.add(L.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          13723400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 32)           9632      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 73, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 32)            4128      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 67, 32)            4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 33, 32)           

In [36]:
history = modelhistory = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=5,
                                             restore_best_weights=True)])

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 11s 505us/step - loss: 0.4295 - acc: 0.7883 - val_loss: 0.3050 - val_acc: 0.8876
Epoch 2/20
22500/22500 [==============================] - 5s 202us/step - loss: 0.3245 - acc: 0.8629 - val_loss: 0.2702 - val_acc: 0.8784
Epoch 3/20
22500/22500 [==============================] - 5s 204us/step - loss: 0.2984 - acc: 0.8756 - val_loss: 0.2566 - val_acc: 0.9108
Epoch 4/20
22500/22500 [==============================] - 5s 201us/step - loss: 0.2808 - acc: 0.8839 - val_loss: 0.3621 - val_acc: 0.8516
Epoch 5/20
22500/22500 [==============================] - 5s 209us/step - loss: 0.2606 - acc: 0.8935 - val_loss: 0.4297 - val_acc: 0.8220
Epoch 6/20
22500/22500 [==============================] - 5s 206us/step - loss: 0.2479 - acc: 0.8999 - val_loss: 0.3316 - val_acc: 0.8732
Epoch 7/20
22500/22500 [==============================] - 5s 207us/step - loss: 0.2248 - acc: 0.9092 - val_loss: 0.1599 - 

Accuracy на валидационном датасете: 0.93


In [37]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 2s 62us/step


[0.35912604201972487, 0.86176]

Accuracy на тестовом датасете: 0.86
### CNN model with batch normalization


In [0]:
reset_tf_session()

In [39]:
model = Sequential()
model.add(L.Embedding(vocab_size, 100, weights=[embedding_matrix],
                      input_length=150, trainable=False))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))

model.add(L.Flatten())
model.add(L.BatchNormalization())
model.add(L.Dense(128, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dense(64, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          13723400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 32)           9632      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 73, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 32)            4128      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 67, 32)            4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 33, 32)           

In [40]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=5,
                                             restore_best_weights=True)])

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 8s 376us/step - loss: 0.4347 - acc: 0.7961 - val_loss: 0.0764 - val_acc: 0.9792
Epoch 2/20
22500/22500 [==============================] - 8s 345us/step - loss: 0.3464 - acc: 0.8516 - val_loss: 0.1106 - val_acc: 0.9700
Epoch 3/20
22500/22500 [==============================] - 8s 353us/step - loss: 0.3188 - acc: 0.8641 - val_loss: 0.2347 - val_acc: 0.9084
Epoch 4/20
22500/22500 [==============================] - 8s 355us/step - loss: 0.3022 - acc: 0.8728 - val_loss: 0.1334 - val_acc: 0.9572
Epoch 5/20
22500/22500 [==============================] - 8s 348us/step - loss: 0.2805 - acc: 0.8821 - val_loss: 0.7373 - val_acc: 0.6628
Epoch 6/20
22500/22500 [==============================] - 8s 351us/step - loss: 0.2653 - acc: 0.8919 - val_loss: 0.2352 - val_acc: 0.9064


Accuracy на валидационном датасете: 0.98

In [41]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 2s 76us/step


[0.6183812316471339, 0.74144]

Accuracy на тестовом датасете: 0.74
### CNN model with dropout rate = 0.1

In [0]:
reset_tf_session()

In [43]:
model = Sequential()
model.add(L.Embedding(vocab_size, 100, weights=[embedding_matrix],
                      input_length=150, trainable=False))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))

model.add(L.Flatten())
model.add(L.Dropout(0.1))
model.add(L.Dense(128, activation='relu'))
model.add(L.Dropout(0.1))
model.add(L.Dense(64, activation='relu'))
model.add(L.Dropout(0.1))
model.add(L.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          13723400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 32)           9632      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 73, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 32)            4128      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 67, 32)            4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 33, 32)           

In [44]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=5,
                                             restore_best_weights=True)])

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 6s 253us/step - loss: 0.4287 - acc: 0.7992 - val_loss: 0.5177 - val_acc: 0.7988
Epoch 2/20
22500/22500 [==============================] - 5s 231us/step - loss: 0.3386 - acc: 0.8568 - val_loss: 0.5754 - val_acc: 0.7304
Epoch 3/20
22500/22500 [==============================] - 5s 235us/step - loss: 0.3081 - acc: 0.8724 - val_loss: 0.4510 - val_acc: 0.8040
Epoch 4/20
22500/22500 [==============================] - 5s 230us/step - loss: 0.2855 - acc: 0.8819 - val_loss: 0.3133 - val_acc: 0.8668
Epoch 5/20
22500/22500 [==============================] - 5s 225us/step - loss: 0.2688 - acc: 0.8902 - val_loss: 0.7628 - val_acc: 0.7040
Epoch 6/20
22500/22500 [==============================] - 5s 228us/step - loss: 0.2485 - acc: 0.8988 - val_loss: 0.2532 - val_acc: 0.8964
Epoch 7/20
22500/22500 [==============================] - 5s 234us/step - loss: 0.2265 - acc: 0.9117 - val_loss: 0.4495 - v

Accuracy на валидационном датасете: 0.90

In [45]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 1s 60us/step


[0.3294014330339432, 0.8684]

Accuracy на тестовом датасете: 0.87
### CNN model with dropout rate = 0.5

In [0]:
reset_tf_session()

In [47]:
model = Sequential()
model.add(L.Embedding(vocab_size, 100, weights=[embedding_matrix],
                      input_length=150, trainable=False))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))

model.add(L.Flatten())
model.add(L.Dropout(0.5))
model.add(L.Dense(128, activation='relu'))
model.add(L.Dropout(0.5))
model.add(L.Dense(64, activation='relu'))
model.add(L.Dropout(0.5))
model.add(L.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          13723400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 32)           9632      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 73, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 32)            4128      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 67, 32)            4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 33, 32)           

In [48]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=5,
                                             restore_best_weights=True)])

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 5s 238us/step - loss: 0.4858 - acc: 0.7556 - val_loss: 0.6129 - val_acc: 0.7712
Epoch 2/20
22500/22500 [==============================] - 5s 225us/step - loss: 0.3593 - acc: 0.8521 - val_loss: 0.2653 - val_acc: 0.9004
Epoch 3/20
22500/22500 [==============================] - 5s 223us/step - loss: 0.3315 - acc: 0.8668 - val_loss: 0.3013 - val_acc: 0.8548
Epoch 4/20
22500/22500 [==============================] - 5s 222us/step - loss: 0.3079 - acc: 0.8768 - val_loss: 0.3254 - val_acc: 0.8652
Epoch 5/20
22500/22500 [==============================] - 5s 222us/step - loss: 0.2942 - acc: 0.8838 - val_loss: 0.1951 - val_acc: 0.9180
Epoch 6/20
22500/22500 [==============================] - 5s 219us/step - loss: 0.2722 - acc: 0.8932 - val_loss: 0.3088 - val_acc: 0.8676
Epoch 7/20
22500/22500 [==============================] - 5s 221us/step - loss: 0.2572 - acc: 0.9003 - val_loss: 0.1343 - v

Accuracy на валидационном датасете: 0.95

In [49]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 1s 58us/step


[0.40787848825901746, 0.84504]

Accuracy на тестовом датасете: 0.85
### CNN model with dropout rate = 0.8


In [0]:
reset_tf_session()

In [51]:
model = Sequential()
model.add(L.Embedding(vocab_size, 100, weights=[embedding_matrix],
                      input_length=150, trainable=False))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))

model.add(L.Flatten())
model.add(L.Dropout(0.8))
model.add(L.Dense(128, activation='relu'))
model.add(L.Dropout(0.8))
model.add(L.Dense(64, activation='relu'))
model.add(L.Dropout(0.8))
model.add(L.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          13723400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 32)           9632      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 73, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 32)            4128      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 67, 32)            4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 33, 32)           

In [52]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=5,
                                             restore_best_weights=True)])

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 5s 235us/step - loss: 0.6858 - acc: 0.5458 - val_loss: 0.7434 - val_acc: 0.0000e+00
Epoch 2/20
22500/22500 [==============================] - 5s 219us/step - loss: 0.5461 - acc: 0.7442 - val_loss: 0.6430 - val_acc: 0.8616
Epoch 3/20
22500/22500 [==============================] - 5s 221us/step - loss: 0.4669 - acc: 0.8086 - val_loss: 0.5781 - val_acc: 0.8308
Epoch 4/20
22500/22500 [==============================] - 5s 223us/step - loss: 0.4332 - acc: 0.8278 - val_loss: 0.4920 - val_acc: 0.8708
Epoch 5/20
22500/22500 [==============================] - 5s 222us/step - loss: 0.4108 - acc: 0.8312 - val_loss: 0.3748 - val_acc: 0.9180
Epoch 6/20
22500/22500 [==============================] - 5s 221us/step - loss: 0.3910 - acc: 0.8418 - val_loss: 0.3666 - val_acc: 0.9244
Epoch 7/20
22500/22500 [==============================] - 5s 223us/step - loss: 0.3809 - acc: 0.8458 - val_loss: 0.3048

Accuracy на валидационном датасете: 0.94


In [53]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 2s 60us/step


[0.3797564870452881, 0.8316]

Accuracy на тестовом датасете: 0.83
### CNN model with batch normalization and dropout with the best rate


In [0]:
reset_tf_session()

In [59]:
model = Sequential()
model.add(L.Embedding(vocab_size, 100, weights=[embedding_matrix],
                      input_length=150, trainable=False))

model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 3, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 4, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.Conv1D(32, 5, activation='relu',  kernel_initializer='he_uniform'))
model.add(L.MaxPooling1D(pool_size=2))

model.add(L.Flatten())
model.add(L.BatchNormalization())
model.add(L.Dropout(0.1))
model.add(L.Dense(128, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.1))
model.add(L.Dense(64, activation='relu'))
model.add(L.BatchNormalization())
model.add(L.Dropout(0.1))
model.add(L.Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          13723400  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 148, 32)           9632      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 146, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 73, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 70, 32)            4128      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 67, 32)            4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 33, 32)           

In [60]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=20,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_acc', patience=5,
                                             restore_best_weights=True)])

Train on 22500 samples, validate on 2500 samples
Epoch 1/20
22500/22500 [==============================] - 9s 397us/step - loss: 0.4997 - acc: 0.7537 - val_loss: 0.1604 - val_acc: 0.9436
Epoch 2/20
22500/22500 [==============================] - 8s 376us/step - loss: 0.3634 - acc: 0.8433 - val_loss: 0.2683 - val_acc: 0.8840
Epoch 3/20
22500/22500 [==============================] - 8s 377us/step - loss: 0.3267 - acc: 0.8607 - val_loss: 1.0234 - val_acc: 0.5028
Epoch 4/20
22500/22500 [==============================] - 9s 381us/step - loss: 0.3062 - acc: 0.8716 - val_loss: 0.3017 - val_acc: 0.8876
Epoch 5/20
22500/22500 [==============================] - 9s 380us/step - loss: 0.2924 - acc: 0.8786 - val_loss: 0.0126 - val_acc: 0.9968
Epoch 6/20
22500/22500 [==============================] - 8s 375us/step - loss: 0.2745 - acc: 0.8853 - val_loss: 0.1495 - val_acc: 0.9476
Epoch 7/20
22500/22500 [==============================] - 9s 381us/step - loss: 0.2525 - acc: 0.8959 - val_loss: 0.2347 - v

Accuracy на валидационном датасете: 0.99

In [61]:
model.evaluate(X_test, y_test)

25000/25000 [==============================] - 2s 77us/step


[1.7455725744123758, 0.59836]

Accuracy на тестовом датасете: 0.6 <br>
Переобучилась.
###Conclusion
Самый высокий accuracy score на тестовых данных (0.87) у нейросети, в которой как метод регуляризации используется дропаут с вероятностью 0.1.